In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import pickle
import gzip

In [2]:
FILENAME='mnist.pkl.gz'

def load_mnist(filename):
    return pickle.load(gzip.open(filename, 'rb'), encoding='latin-1')

In [3]:
((x_train, y_train), (x_test, y_test), _) = load_mnist(FILENAME)

In [4]:
x_train.shape , type(x_train) , y_train.shape , type(y_train)

((50000, 784), numpy.ndarray, (50000,), numpy.ndarray)

In [5]:
x_test.shape , type(x_test) , y_test.shape , type(y_test)

((10000, 784), numpy.ndarray, (10000,), numpy.ndarray)

In [6]:
x_train=x_train.reshape([x_train.shape[0],-1])

In [7]:
x_train.shape

(50000, 784)

In [8]:
x_test=x_test.reshape([x_test.shape[0],-1])

In [9]:
x_test.shape

(10000, 784)

In [10]:
lb=LabelBinarizer()

In [11]:
y_train=lb.fit_transform(y_train)

In [12]:
y_test=lb.transform(y_test)

In [13]:
learning_rate = 0.01
training_epochs = 20
batch_size = 64

In [14]:
x = tf.placeholder("float32", [None,784])
y = tf.placeholder("float32", [None, 10])

In [16]:
weights = {
    'h1': tf.Variable(tf.random_normal([28*28,100])),    
    'h2': tf.Variable(tf.random_normal([100, 100])), 
    'out': tf.Variable(tf.random_normal([100, 10]))  
}
biases = {
    'b1': tf.Variable(tf.random_normal([100])),             
    'b2': tf.Variable(tf.random_normal([100])),             
    'out': tf.Variable(tf.random_normal([10]))              
}

In [17]:
def model(x, weights, biases):
    
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
       
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) 
    layer_2 = tf.nn.relu(layer_2)
        
    out_layer = tf.add(tf.matmul(layer_2, weights['out']) , biases['out'])

    return out_layer

In [18]:
predictions = model(x, weights, biases)

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions,labels= y))

In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
for epoch in range(training_epochs):
        arr=np.arange(x_train.shape[0])
        total_cost=0
        np.random.shuffle(arr)
        for i in range(0,x_train.shape[0],batch_size):
            c, _= sess.run([cost,optimizer] , feed_dict={x: x_train[arr[i : i + batch_size]],y: y_train[arr[ i : i + batch_size]]})
            total_cost+=c
        print(total_cost)

15883.81305038929
2096.780656496994
943.5061514851926
498.4930484867655
301.0031564957462
220.2226049862802
180.71937035256997
168.88273962168023
169.34674496855587
155.0635546669364
146.1378858918324
124.67548306542449
124.9043805047404
124.71965398988687
110.03722138956073
95.45931504917098
93.76423000212526
93.88429643784184
102.18345521204174
74.05867120914627


In [24]:
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
a,correct_prediction  = sess.run([predictions, correct_prediction], feed_dict={x:x_test, y:y_test})
correct_prediction.sum()/x_test.shape[0]

0.9646